In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/nba'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [15]:
spark.sql(
    """
    SHOW tables from nba
    """
).show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|     nba|  bz_abrev|      false|
|     nba|bz_players|      false|
|     nba|  bz_teams|      false|
|        |      temp|       true|
+--------+----------+-----------+



In [17]:
#Carregar tabela para dataframe
table_location = "hdfs://hdfs-nn:9000/projeto/nba/bz_abrev"
df = spark.read.format("delta").load(table_location)
df.toPandas()

abreviation                                 name_team          league
0           AKF                 Akron Firestone Non-Skids           (NBL)
1           AKG                  Akron Goodyear Wingfoots          (NBL) 
2           ANA                            Anaheim Amigos           (ABA)
3           AND  Anderson Duffey Packers/Anderson Packers  (NBL/NBA/NPBL)
4           ATC                          Atlanta Crackers         (PBLA) 
..          ...                                       ...             ...
210         WSC                       Washington Capitals      (BAA/NBA) 
211         WSH                           Washington Caps          (ABA) 
212         WSM                        Washington Mystics         (WNBA) 
213         WTP                        Waterloo Pro-Hawks         (PBLA) 
214         YNG                          Youngstown Bears           (NBL)

[215 rows x 3 columns]

In [ ]:
#Codigo para criar view temporaria
    #-#df.createOrReplaceTempView("temp")

In [18]:
#Rename Colunas
df2 = df.withColumnRenamed("abreviation","sigla") \
    .withColumnRenamed("name_team","equipa")\
    .withColumnRenamed("league","liga")
df2.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- equipa: string (nullable = true)
 |-- liga: string (nullable = true)



In [19]:
#Retirar os espaços brancos aos valores da coluna "liga"
from pyspark.sql.functions import trim
df3 = df2.withColumn('liga', trim(df2.liga))

In [20]:
#Verificar se os espaços brancos foram retirados
df3.createOrReplaceTempView("temp")

spark.sql(
"""
select * from temp
where liga == '(NBA)'
"""
).show()

+-----+--------------------+-----+
|sigla|              equipa| liga|
+-----+--------------------+-----+
|  ATL|       Atlanta Hawks|(NBA)|
|  BAL|   Baltimore Bullets|(NBA)|
|  BUF|      Buffalo Braves|(NBA)|
|  CAP|     Capital Bullets|(NBA)|
|  CHA|   Charlotte Hornets|(NBA)|
|  CHN|   Charlotte Bobcats|(NBA)|
|  CHI|       Chicago Bulls|(NBA)|
|  CHP|Chicago Packers/Z...|(NBA)|
|  CIN|   Cincinnati Royals|(NBA)|
|  CLE| Cleveland Cavaliers|(NBA)|
|  DAL|    Dallas Mavericks|(NBA)|
|  DET|     Detroit Pistons|(NBA)|
|  DLC|   Dallas Chaparrals|(NBA)|
|  GST|Golden State Warr...|(NBA)|
|  HOU|     Houston Rockets|(NBA)|
|  INO|Indianapolis Olym...|(NBA)|
|  KCK|   Kansas City Kings|(NBA)|
|  KCO|Kansas City-Omaha...|(NBA)|
|  LAC|Los Angeles Clippers|(NBA)|
|  LAL|  Los Angeles Lakers|(NBA)|
+-----+--------------------+-----+
only showing top 20 rows



In [21]:
#substituir valores na coluna "liga"
from pyspark.sql.functions import when
df4 = df3.withColumn("liga", when(df3.liga == "(NBA)","NBA") \
    .when(df3.liga == "(NBL/NBA/NPBL)","NBA") \
    .when(df3.liga == "(ABL/BAA/NBA)","NBA") \
    .when(df3.liga == "(BAA/NBA)","NBA") \
    .when(df3.liga == "(NBL/NBA/NBPL)","NBA") \
    .when(df3.liga == "(ABA/NBA)","NBA") \
    .when(df3.liga == "(NBL/BAA/NBA)","NBA") \
    .when(df3.liga == "(NBL/NBA)","NBA") \
    .when(df3.liga == "(NBA/NBA includes Buffalo Bisons)","NBA") \
    .otherwise("OUTRA"))
df4.show()

+-----+--------------------+-----+
|sigla|              equipa| liga|
+-----+--------------------+-----+
|  AKF|Akron Firestone N...|OUTRA|
|  AKG|Akron Goodyear Wi...|OUTRA|
|  ANA|      Anaheim Amigos|OUTRA|
|  AND|Anderson Duffey P...|  NBA|
|  ATC|    Atlanta Crackers|OUTRA|
|  ATG|       Atlanta Glory|OUTRA|
|  ATL|       Atlanta Hawks|  NBA|
|  BAC|     Baltimore Claws|OUTRA|
|  BAL|   Baltimore Bullets|  NBA|
|  BFB|      Buffalo Bisons|OUTRA|
|  BFG|     Buffalo Germans|OUTRA|
|  BIR| Birmingham Skyhawks|OUTRA|
|  BKA|  Brooklyn Arcadians|OUTRA|
|  BKC|    Brooklyn Celtics|OUTRA|
|  BKV|Brooklyn Visitations|OUTRA|
|  BLB|   Baltimore Bullets|  NBA|
|  BOS|      Boston Celtics|  NBA|
|  BSW|   Boston Whirlwinds|OUTRA|
|  BTO|   Baltimore Orioles|OUTRA|
|  BUF|      Buffalo Braves|  NBA|
+-----+--------------------+-----+
only showing top 20 rows



In [23]:
#Guardar o ultimo DataFrama (df4) na hivetable de bronze
#Como esta tabela "nba.bz_abrev" é auxiliar depois de refinada não é grava em silver
#Faz-se overwrite na tabela de bronze
df4.select("*")\
    .write\
    .mode("overwrite")\
    .option("overwriteSchema","true")\
    .format("delta")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/bz_abrev")

In [25]:
spark.sql(
"""
select * from nba.bz_abrev
"""
).show()

+-----+--------------------+-----+
|sigla|              equipa| liga|
+-----+--------------------+-----+
|  AKF|Akron Firestone N...|OUTRA|
|  AKG|Akron Goodyear Wi...|OUTRA|
|  ANA|      Anaheim Amigos|OUTRA|
|  AND|Anderson Duffey P...|  NBA|
|  ATC|    Atlanta Crackers|OUTRA|
|  ATG|       Atlanta Glory|OUTRA|
|  ATL|       Atlanta Hawks|  NBA|
|  BAC|     Baltimore Claws|OUTRA|
|  BAL|   Baltimore Bullets|  NBA|
|  BFB|      Buffalo Bisons|OUTRA|
|  BFG|     Buffalo Germans|OUTRA|
|  BIR| Birmingham Skyhawks|OUTRA|
|  BKA|  Brooklyn Arcadians|OUTRA|
|  BKC|    Brooklyn Celtics|OUTRA|
|  BKV|Brooklyn Visitations|OUTRA|
|  BLB|   Baltimore Bullets|  NBA|
|  BOS|      Boston Celtics|  NBA|
|  BSW|   Boston Whirlwinds|OUTRA|
|  BTO|   Baltimore Orioles|OUTRA|
|  BUF|      Buffalo Braves|  NBA|
+-----+--------------------+-----+
only showing top 20 rows

